# Adding an Item with the Omeka S API

The "basics" of accessing the API, including authentication and querying item lists, are covered in the previous notebook
[`omeka-s-api-basics.ipynb`](omeka-s-api-basics.ipynb).

For some (dated 2019) advice on doing this with `curl` see: https://forum.omeka.org/t/example-api-usage-using-curl/8083. 

Since this is a standalone notebook, go through the setup and authentication again:

In [ ]:
import requests
import json

In [ ]:
# establish API location/endpoints

siteUrl = 'http://jajohnst.si676.si.umich.edu/omeka-s' # if you replicate this example, provide the URI for your site
endpoint = '/api'

In [ ]:
def get_credentials(credential_file_path):
    '''Retrieve Omeka S Api credentials from another file. 
    That file must be a JSON file.'''

    with open(credential_file_path, 'r') as credentials:
        keys = json.load(credentials)
    
    return keys['key_identity'], keys['key_credential']

key_identity, key_credential = get_credentials('../collection-site-materials/omeka-credentials.json')

In [ ]:
print(key_credential, key_identity)

## Use the Omeka S API to add a new item

The process involves the data for the new item, then
using a `post` request to add the item via the API. 

In [ ]:
action = '/items'

parameters = {
    'key_credential': key_credential,
    'key_identity'  : key_identity
}

headers = {
    'Content-Type': 'application/json'
}

There are various ways to go about this, and this is not the most JSON/pythonic
way to do it. The below provides a customized template by hand, but
a better production workflow could automate this from an item feeder list,
an API response, or another dynamic method. 

In [ ]:
# create data for a new item

data = { 
    "dcterms:title": [{
        "property_id": 1,
        "property_label": "Title",
        "@value": "A Mere Title for an item created via the API",
        "type": "literal",
        "@language": "en-us"
        }],
    "dcterms:rights": [{
        "type": "literal",
        "property_id": 15,
        "property_label": "Rights",
        "is_public": True,
        "@value": "No known restrictions on publication.",
        "@language": "en-us"
        }]
    }

In [ ]:
# now, post the data

post_item_url = siteUrl + endpoint + action

new_item_post = requests.post(post_item_url, headers=headers, params=parameters, data=json.dumps(data))

print(new_item_post.url,new_item_post.status_code,'\n',new_item_post.headers)

In [ ]:
new_item_post.json()

## Use the Omeka S API to add a new item with Media

The process requires the metadata for the new item,
the link to the file (aka "media" in Omeka S terms), and
then using a `post` request to add the item via the API. 

In [ ]:
# example of how you can format the metadata
# note that this is a python dictionary, and can then be converted in valid JSON

post_data = {
    "dcterms:title": [
        {
            "property_id": 1,
            "property_label": "Title",
            "@value": "An image of an Orca, archived from an old website, and uploaded via the API",
            "type": "literal"
        }
    ],
    "@type": "o:Item",
    "o:media": [
        {
            "o:ingester": "upload",
            "file_index": "0",
            "o:item": {},
            "dcterms:title": [
                {
                    "property_id": 1,
                    "property_label": "Title",
                    "@value": "Media file for the archived Orca image",
                    "type": "literal"
                }
            ]
        }
    ]
}

### Uploading the media item with Python requests

The next block creates a URL for the `/items` action of the API,
sets authentication information to be included in the URL as parameters,
and finally establishes a dictionary called `files`, which contains
the item metadata as `data`, and a corresponding `file[0]` that is matched to this item data.
Finally, requests sends this data as a POST request with request's `.post()` function.

In [ ]:
siteAPI = 'http://jajohnst.si676.si.umich.edu/omeka-s/api'
action = '/items'

parameters = {
    'key_credential': key_credential,
    'key_identity'  : key_identity
}

files = {
    'data': (None, json.dumps(post_data)),
    'file[0]': open('../data/orca.via_.moc_.noaa_.jpg', 'rb'),
}

r = requests.post(siteAPI + action, params=parameters, files=files)

print(r.url, '\n', r.status_code)

In [ ]:
r.json()

## Using `curl` instead of requests

See these examples of using the Omeka S API with `curl`: https://forum.omeka.org/t/example-api-usage-using-curl/8083

In [ ]:
post_data = { "dcterms:title" : [ {"property_id": 1, "property_label" : "Title", "@value" : "An image of an Orca, archived from an old website, and uploaded via the API", "type" : "literal" } ], "@type" : "o:Item", "o:item_set" : [ {"o:id": 440}], "o:media" : [{"o:ingester": "upload", "file_index": "0", "o:item": {}, "dcterms:title" : [ { "property_id" : 1, "property_label" : "Title", "@value" : "Media file for the archived Orca image", "type" : "literal" } ]}] }

print(json.dumps(post_data)) # this converts the data into valid JSON

### Working `curl` request

```bash
curl -v -F 'data={"dcterms:title": [{"property_id": 1, "property_label": "Title", "@value": "An image of an Orca, archived from an old website, and uploaded via the API", "type": "literal"}], "@type": "o:Item", "o:item_set": [{"o:id": 441}], "o:media": [{"o:ingester": "upload", "file_index": "0", "o:item": {}, "dcterms:title": [{"property_id": 1, "property_label": "Title", "@value": "Media file for the archived Orca image", "type": "literal"}]}]}' -F "file[0]=@./data/webfiles-samples/image/orca.via_.moc_.noaa_.jpg" "http://jajohnst.si676.si.umich.edu/omeka-s/api/items?key_credential=nj0K6JAqnFgeqyP3Kq40SOhl6fiPBXky&key_identity=yAu5NAEEtJCvGx4s8XgJSgvTFjwvLD6j"
```

Using the working `curl` command, trying this curl converter to python requests... 

https://curlconverter.com/